# Generate tiled region run file

## This notebook is an example: create a copy before running it or you will get merge conflicts!

**NOTE**: Before running this notebook for the first time, make sure you've coregistered your instrument using the *update coregistration parameters* section of  `1_set_up_toffy.ipynb`. This will ensure your FOVs display correctly on the slide.

This notebook automatically generates FOVs across multiple tiled regions on a sample and allows you to remove unneeded FOVs created during the process.

In [2]:
import sys
sys.path.append('../')

In [3]:
import json
import os
from skimage.io import imread

from toffy.json_utils import read_json_file, write_json_file, split_run_file
from toffy import tiling_utils

# suppress mpl deprecation
import warnings
from matplotlib.cbook import mplDeprecation
warnings.filterwarnings("ignore", category=mplDeprecation)

## Define the data paths

### Set *json_tiling_dir*, the name of the directory to place and write all of your files. This directory should be placed in *C:\\\Users\\\Customer.ION\\\Documents\\\tiled_run_jsons*.

### After running the data paths section, the *json_tiling_dir* directory structure should look like:

- `C:\\Users\\Customer.ION\\Documents\\tiled_run_jsons`
    - `json_tiling_dir`
        - `region_corners_path` (a JSON file defining the position and name of each region to tile)
        - `slide_path` (a PNG or BMP file containing the slide you want to tile over)
        - `moly_path` (an optional JSON file defining a Moly point to separate FOVs in the run file)

In [ ]:
# define the name of the directory to use in tiled_run_jsons
json_tiling_dir = 'YYYY-MM-DD_test'

In [ ]:
# convert the name to an absolute path, DO NOT CHANGE THIS
json_tiling_dir = os.path.join('C:\\Users\\Customer.ION\\Documents\\tiled_run_jsons', json_tiling_dir)

### Define a name prefix to use for the input and output files to place in *json_tiling_dir*. **All files must conform to this naming scheme**.

In [ ]:
# define the prefix for each file
tiled_region_prefix = 'example_tiled_region'

### To identify each desired tiled region on the image, you must provide a region_corners_path that defines the top-left corner FOV for each of them.

### To generate these FOVs, use the Mibitracker to select the top-left FOV for each desired tiled region. Name each FOV to the corresponding desired name of the tiled region it defines, then download the file (in JSON format). *region_corners_path* should be formatted just like a commercial instrument run file, with each top-left FOV contained in the *'fovs'* key. Place this file directly into the path specified in *json_tiling_dir*. Ensure this file is renamed *{tiled_region_prefix}_region_corners.json* (ex. if you set *tiled_region_prefix = test* in the notebook, the file should be renamed *test_tiled_region_corners.json*).

In [ ]:
region_corners_path = os.path.join(json_tiling_dir, '%s_region_corners.json' % tiled_region_prefix) 

### Place *slide_path*, the optical image file containing the slide to visualize the tiled regions over, directly in *json_tiling_dir*. The image should be saved in *.png* or *.bmp* format.

In [ ]:
slide_path = os.path.join(json_tiling_dir, "%s_slide.png" % tiled_region_prefix)

### Place *moly_path*, the JSON file defining a Moly point, directly in *json_tiling_dir*. The Moly point JSON file should also be downloaded off of the Mibitracker.

### This is needed if you want to insert Moly points at certain intervals between FOVs for calibration purposes.

In [ ]:
moly_path = os.path.join(json_tiling_dir, '%s_moly_point.json' % tiled_region_prefix)

### Define *tiled_region_path*, the output file in *json_tiling_dir* to write the final set of FOVs for each tiled region after the notebook is complete.

In [ ]:
tiled_region_path = os.path.join(json_tiling_dir, "%s_tiled_region_mapping.json" % tiled_region_prefix)

## Set tiling parameters

### Run following cell, which will ask you to enter additional info for each specified region in *region_corners_path*. This will include the number of FOVs along the row and column axes, whether to randomize the order of FOVs in a tiled region, whether to insert Moly points between tiled regions, and whether to insert Moly points between FOVs at a specified interval. 

In [ ]:
tiling_params = tiling_utils.set_tiled_region_params(region_corners_path)

## Create the FOVs for each region

### FOV names correspond to their row and column position on the grid of FOVs and are 1-indexed. For example, *R1C1* means the FOV is in row 1 and column 1, *R2C1* would be the next FOV down, and *R1C2* would be the next FOV to the right. The top left FOV is *R1C1*, the bottom right FOV would be *R4C5* for a tiled region of 4x5 dimensions. 

In [ ]:
# generate the FOVs in each region
tiled_region_fovs = tiling_utils.generate_tiled_region_fov_list(
    tiling_params,
    moly_path
)

## Interactive FOV deselection

### The grid defining an ROI may cover more area than you wish to tile. This GUI allows you to interactively delete FOVs from *tiled_region_fovs* prior to saving:

### 1. Click on a FOV to propose it for removal. The border of proposed FOVs will be highlighted. Click again to unpropose the FOV.
### 2.Once you've selected all the FOVs you want to remove, click *Delete selected FOVs* to remove them. **This action cannot be undone!**
### 3. You can always remove more FOVs even after saving. Just remember to re-save.
### 4. After you're done removing all the FOVs, click *Save mapping* to save your truncated set of FOVs to *tiled_region_path*.

In [ ]:
# read the slide image in
slide_data = imread(slide_path)

In [ ]:
%matplotlib widget
_ = tiling_utils.tiled_region_interactive_remap(
    tiled_region_fovs,
    tiling_params,
    slide_data,
    tiled_region_path,
    figsize=(7, 7)
)

### Run JSON adjustment

#### If you would like to break up your tiled file into smaller JSONs containing a specified amount of FOVs in each, you can do that using the code below. 

#### `file_split` is a list of values detailing how many FOVs to included in each new file, and must sum to the total number of FOVs in the run file.

In [5]:
file_split = []

split_run_file(json_tiling_dir, os.path.basename(tiled_region_fovs_path), file_split)